In [43]:
import requests
import re
import urllib.request
from bs4 import BeautifulSoup
from collections import deque
from html.parser import HTMLParser
from urllib.parse import urlparse
import os

In [33]:
!pip install bs4

## openai 웹사이트 크롤링 하기. 우리는 각자 데이터 수집을 했으니 이 과정이 없어도 된다. 튜토리얼이므로 따라하는 것에 의미를 두자.

In [44]:
# Regex pattern to match a URL
HTTP_URL_PATTERN = r'^http[s]*://.+'

# Define root domain to crawl
domain = "openai.com"
full_url = "https://openai.com/"

# Create a class to parse the HTML and get the hyperlinks
class HyperlinkParser(HTMLParser):
    def __init__(self):
        super().__init__()
        # Create a list to store the hyperlinks
        self.hyperlinks = []

    # Override the HTMLParser's handle_starttag method to get the hyperlinks
    def handle_starttag(self, tag, attrs):
        attrs = dict(attrs)

        # If the tag is an anchor tag and it has an href attribute, add the href attribute to the list of hyperlinks
        if tag == "a" and "href" in attrs:
            self.hyperlinks.append(attrs["href"])

In [46]:
# Function to get the hyperlinks from a URL
def get_hyperlinks(url):
    
    # Try to open the URL and read the HTML
    try:
        # Open the URL and read the HTML
        with urllib.request.urlopen(url) as response:

            # If the response is not HTML, return an empty list
            if not response.info().get('Content-Type').startswith("text/html"):
                return []
            
            # Decode the HTML
            html = response.read().decode('utf-8')
    except Exception as e:
        print(e)
        return []

    # Create the HTML Parser and then Parse the HTML to get hyperlinks
    parser = HyperlinkParser()
    parser.feed(html)

    return parser.hyperlinks

In [37]:
# Function to get the hyperlinks from a URL that are within the same domain
def get_domain_hyperlinks(local_domain, url):
    clean_links = []
    for link in set(get_hyperlinks(url)):
        clean_link = None

        # If the link is a URL, check if it is within the same domain
        if re.search(HTTP_URL_PATTERN, link):
            # Parse the URL and check if the domain is the same
            url_obj = urlparse(link)
            if url_obj.netloc == local_domain:
                clean_link = link

        # If the link is not a URL, check if it is a relative link
        else:
            if link.startswith("/"):
                link = link[1:]
            elif link.startswith("#") or link.startswith("mailto:"):
                continue
            clean_link = "https://" + local_domain + "/" + link

        if clean_link is not None:
            if clean_link.endswith("/"):
                clean_link = clean_link[:-1]
            clean_links.append(clean_link)

    # Return the list of hyperlinks that are within the same domain
    return list(set(clean_links))

In [38]:
def crawl(url):
    # Parse the URL and get the domain
    local_domain = urlparse(url).netloc

    # Create a queue to store the URLs to crawl
    queue = deque([url])

    # Create a set to store the URLs that have already been seen (no duplicates)
    seen = set([url])

    # Create a directory to store the text files
    if not os.path.exists("text/"):
            os.mkdir("text/")

    if not os.path.exists("text/"+local_domain+"/"):
            os.mkdir("text/" + local_domain + "/")

    # Create a directory to store the csv files
    if not os.path.exists("processed"):
            os.mkdir("processed")

    # While the queue is not empty, continue crawling
    while queue:

        # Get the next URL from the queue
        url = queue.pop()
        print(url) # for debugging and to see the progress

        # Save text from the url to a <url>.txt file
        with open('text/'+local_domain+'/'+url[8:].replace("/", "_") + ".txt", "w") as f:

            # Get the text from the URL using BeautifulSoup
            soup = BeautifulSoup(requests.get(url).text, "html.parser")

            # Get the text but remove the tags
            text = soup.get_text()

            # If the crawler gets to a page that requires JavaScript, it will stop the crawl
            if ("You need to enable JavaScript to run this app." in text):
                print("Unable to parse page " + url + " due to JavaScript being required")
            
            # Otherwise, write the text to the file in the text directory
            f.write(text)

        # Get the hyperlinks from the URL and add them to the queue
        for link in get_domain_hyperlinks(local_domain, url):
            if link not in seen:
                queue.append(link)
                seen.add(link)

In [39]:
def remove_newlines(serie):
    serie = serie.str.replace('\n', ' ')
    serie = serie.str.replace('\\n', ' ')
    serie = serie.str.replace('  ', ' ')
    serie = serie.str.replace('  ', ' ')
    return serie

In [40]:
# 실제로는 시간 최소 20분 이상 걸리니 마음의 준비를 단단히 할 것.
# 우리는 실습 목적이므로 3분 정도만 돌려보고 크롤링을 중단한다.
crawl(full_url)

https://openai.com/


UnicodeEncodeError: 'cp949' codec can't encode character '\xc2' in position 124: illegal multibyte sequence

In [41]:
import pandas as pd

# 텍스트 파일들 리스트로 만들기
texts=[]

# 모든 텍스트 파일 가져와서 내용 리스트에 붙이기
for file in os.listdir("text/" + domain + "/"):

    # Open the file and read the text
    with open("text/" + domain + "/" + file, "r") as f:
        text = f.read()

        # Omit the first 11 lines and the last 4 lines, then replace -, _, and #update with spaces.
        texts.append((file[11:-4].replace('-',' ').replace('_', ' ').replace('#update',''), text))

# 파일 경로와 텍스트로 데이터 프레임 만들기
df = pd.DataFrame(texts, columns = ['fname', 'text'])

ModuleNotFoundError: No module named 'pandas'

In [42]:
# 개행문자 삭제해서 파일 이름과 함께 붙여서 text 칼럼에 넣기.
df['text'] = df.fname + ". " + remove_newlines(df.text)
df.to_csv('processed/scraped.csv')
df.head()

NameError: name 'df' is not defined

## 아래는 텍스트 임베딩을 가져오는데 소요되는 전체 토큰을 계산하는 코드이다. 

In [ ]:
import tiktoken
df = pd.read_csv('processed/scraped.csv', index_col=0)
df.columns = ['title', 'text']

#  cl100k_base는 토크나이저다. 임베딩과는 큰 관련이 없다.
tokenizer = tiktoken.get_encoding("cl100k_base")

# 소요되는 토큰을 각 칼럼에 저장하기. apply 함수가 쓰였다!
df['n_tokens'] = df.text.apply(lambda x: len(tokenizer.encode(x)))


In [ ]:

# 히스토그램으로 그려서 보기
df.n_tokens.hist()

## 텍스트가 최대 입력 토큰을 초과할 수도 있다. 그러므로, 최대 입력 길이만큼 쪼개주는 함수를 만들어야 한다.
## text-ada-002의 경우, 최대 입력 길이가 4095이다.

In [ ]:
max_tokens =500
def split_into_many(text, max_tokens = max_tokens):

    # 일단 문장으로 쪼개기
    sentences = text.split('. ')

    # 문장마다 몇 토큰 들어가는지 세어 주기.
    n_tokens = [len(tokenizer.encode(" " + sentence)) for sentence in sentences]
    
    chunks = []
    tokens_so_far = 0
    chunk = []


    for sentence, token in zip(sentences, n_tokens):

        # 문장의 토큰 수와 현재까지의 토큰 수의 합이 최대 토큰 수를 초과하면, 해당 청크를 청크 목록에 추가하고 청크 및 현재까지의 토큰 수를 초기화한다.
        if tokens_so_far + token > max_tokens:
            chunks.append(". ".join(chunk) + ".")
            chunk = []
            tokens_so_far = 0


        if token > max_tokens:
            continue

        chunk.append(sentence)
        tokens_so_far += token + 1

    if chunk:
        chunks.append(". ".join(chunk) + ".")

    return chunks

In [ ]:
shortened = []
# 데이터 프레임 순회하기
for row in df.iterrows():

    # If the text is None, go to the next row
    if row[1]['text'] is None:
        continue

    # If the number of tokens is greater than the max number of tokens, split the text into chunks
    if row[1]['n_tokens'] > max_tokens:
        shortened += split_into_many(row[1]['text'])
    
    # Otherwise, add the text to the list of shortened texts
    else:
        shortened.append( row[1]['text'] )

In [ ]:
df = pd.DataFrame(shortened, columns = ['text'])
df['n_tokens'] = df.text.apply(lambda x: len(tokenizer.encode(x)))
df.n_tokens.hist()

In [ ]:
import os
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv('../.env'))
from openai import OpenAI
import openai
client = OpenAI()
openai.api_key  = os.getenv('OPENAI_API_KEY')

In [ ]:
df['embeddings'] = df.text.apply(lambda x: client.embeddings.create(input=x, model='text-embedding-ada-002').data[0].embedding)
df.to_csv('processed/embeddings.csv')
df.head()

In [ ]:
import pandas as pd
import numpy as np
from scipy.spatial.distance import cosine

df=pd.read_csv('processed/embeddings.csv', index_col=0)
df['embeddings'] = df['embeddings'].apply(eval).apply(np.array)
df.head()

In [ ]:
def create_context(
    question, df, max_len=1800
):
    """
    데이터 프레임으로부터 가장 유사한 문맥을 찾아서 질문에 대한 문맥을 만든다.
    """

    # 질문으로부터 임베딩을 만든다.
    q_embeddings = client.embeddings.create(input=question, model='text-embedding-ada-002').data[0].embedding

    # 임베딩으로부터 거리를 계산한다.
    df["distances"] = df["embeddings"].apply(lambda x: cosine(q_embeddings, x))
    returns = []
    cur_len = 0
    # 거리순으로 정렬하고, 문맥이 너무 길어지기 전까지만 텍스트를 문맥에 추가한다.
    for i, row in df.sort_values('distances', ascending=True).iterrows():
        

        # 현재 토큰 수에 텍스트 길이를 더한다.
        cur_len += row['n_tokens'] + 4
        
        # 문맥이 너무 길어지면, break
        if cur_len > max_len:
            break
        

        # 아니면, 문맥에 추가한다.
        returns.append(row["text"])

    # Return the context
    return "\n\n###\n\n".join(returns)


# 이 함수가 하는 일 다시 정리해보자.
# 1. 질문이 들어오면 질문을 임베딩한다. 즉, 숫자 표현으로 바꿔준다.
# 2. 모든 문장들의 임베딩과 질문의 임베딩 사이의 거리를 계산한다.
# 3. 문맥이 너무 길어지면 안되니까, 가장 거리가 가까운 문장부터 차례대로 문맥에 추가한다.
# 4. 문맥을 반환한다.

In [ ]:
def answer_question(
    df,
    model="gpt-3.5-turbo",
    question="Am I allowed to publish model outputs to Twitter, without a human review?",
    max_len=1800,
    debug=False,
):
    """
    데이터 프레임 텍스트에서 가장 유사한 문맥을 기반으로 질문에 답한다.
    """
    context = create_context(
        question,
        df,
        max_len=max_len,
    )

    if debug:
        print("Context:\n" + context)
        print("\n\n")

    try:
        response = client.chat.completions.create(
            model=model,
            messages=[
                {"role": "system", "content": "Answer the question based on the context below, and if the question can't be answered based on the context, say \"I don't know\"\n\n"},
                {"role": "user", "content": f"Context: {context}\n\n---\n\n Question: {question}"}
            ],
            temperature=0,
        )
        return response.choices[0].message.content
    except Exception as e:
        print("Error occurred:", e)
        return ""

In [ ]:
answer_question(df, question="What day is it?", debug=False)

In [ ]:
answer_question(df, question="What is newest embeddings model?", debug=False)

In [ ]:
answer_question(df, question="How much does it cost to use gpt-4?", debug=False)

In [ ]:
answer_question(df, question="최신 임베딩 모델이 뭐야?", debug=False)

In [ ]:
answer_question(df, question="What is newest embeddings model? 한국말로 응답해줘", debug=False)